In [132]:
# ML modules
import pandas as pd
import numpy as np
import seaborn as sns

#GraphQl
from gql import gql, Client
from gql.dsl import DSLQuery, DSLSchema, dsl_gql
from gql.transport.requests import RequestsHTTPTransport

#API Key
from dotenv import load_dotenv
import os

# Set up logging
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [15]:
load_dotenv()
# Add your key and endpoint
key = os.getenv('API_KEY')

client_id = os.getenv('CLIENT_ID')

url_yelp = 'https://api.yelp.com/v3/graphql'

# YELP API Request

In [122]:
header = {
    'Authorization': 'bearer {}'.format(key),
    'Content-Type':"application/json"
}

transport = RequestsHTTPTransport(url = url_yelp, #Yelp.config('url'),
                                  headers = header,
                                  use_json = True)

client = Client(transport = transport, fetch_schema_from_transport = True)

cities = ['Paris', 'Lyon']

#'Paris', 'Bordeaux', 
#           'Lyon', 'Vannes',
#           'Marseille', 'Nice',
#           'Lille', 'Grenoble',
#           'Cannes', 'Limoges',
#           'Versailles', 'Nice',
#           'Nantes']

In [130]:
results = []
#query RequestForEachCity($city: String!) {

for city in cities :
    query = gql('''{
                search(term : "restaurant", location : "$city", limit : 50) {
                    business{
                        name
                        id
                        reviews {
                            id
                            text
                            rating
                        }
                    }
               }
            }''')
    
    #params = {'city': city}
    results.append(client.execute(query))#, variable_values=params))

In [124]:
dict_reviews = []

for res in results:
    for val in res['search']['business']:
        for reviews in val['reviews']:
            dict_reviews.append({
                'review_id' : reviews['id'],
                'text' : reviews['text'],
                'stars' : reviews['rating']
            })

In [125]:
df_yelp = pd.DataFrame.from_dict(dict_reviews)
#df_yelp = df_yelp.dropna()

In [131]:
df_yelp

,review_id,text,stars
0,sMG5b4TBAsIBi5wb4-OsEA,Simply fantastic! The lengthy wait was well wo...,5
1,_q8gqR-4DGKQTHUmlL0jug,For the ambiance and the service I give this p...,3
2,_tPpqYH3mH-3OvWWdlp4Wg,Who doesn't love pasta?\n\nThe queue here gets...,5
3,93nmdQDiWOFhGK_K6AzXzw,The drinks are just phenomenal. Food was delic...,5
4,D5QCC3tC4nelJvfPQNANhQ,"Good experience, theatrical drinks that were a...",4
...,...,...,...
280,pKUW3lCVt2pi1mB4yrx3Lw,This is my go choo Korean restaurant in Stockh...,5
281,JkCCJCvHG6uMtdErda3flg,Great little cozy place! Good portions and rea...,5
282,73ghH96uT4HuquT8Y7N6Jg,Wow. Great food and big portions to a very low...,4
283,5YPWGQdldotWF9C2G0hqKw,Amazing food at an affordable price. The small...,5


In [133]:
if ~os.path.exists("data/processed/reviews_from_api.pkl.gz"):
    df_yelp.to_pickle("data/processed/reviews_from_api.pkl.gz")
    logger.info(">>> Saving reviews to pickle file that was charged from API...")

INFO:__main__:>>> Saving reviews to pickle file that was charged from API...
